In [1]:
import pandas as pd

In [2]:
df_thread = pd.read_csv('thread_18cff0b946f3a4d4_cleaned.csv', index_col='Unnamed: 0')
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 76 to 100
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   threadId          11 non-null     object
 1   from              11 non-null     object
 2   date              11 non-null     object
 3   body_cleaned      11 non-null     object
 4   body_cleaned_len  11 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [3]:
df_thread = df_thread.reset_index().drop('index', axis=1)
df_thread

,threadId,from,date,body_cleaned,body_cleaned_len
0,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 15:59:44-03:00,"Prezada DGBD, boa tarde!\r\n\r\nEstamos com er...",105
1,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 16:29:16-03:00,Copiando para o e-mail correto da DIBD.\r\n\r\...,15
2,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 17:09:03-03:00,Parece ser de tnsnames.ora desatualizado na es...,36
3,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:29:30-03:00,"Obrigado, Edson.",2
4,18cff0b946f3a4d4,Evandro Bassanesi <evandro.bassanesi@trt4.jus.br>,2024-01-12 17:34:50-03:00,"Felipe,\r\n\r\nEste está desatualizado. O banc...",24
5,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:46:45-03:00,Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencios...,18
6,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 08:58:54-03:00,"Felipe, isso não ocorre no servidor do Markvis...",104
7,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 09:19:39-03:00,"Agora que entendi, o problema que refere, não...",46
8,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 10:18:12-03:00,A GPO de atualização do TNSnames (desktop-all-...,28
9,18cff0b946f3a4d4,Francisco Flesch <francisco.flesch@trt4.jus.br>,2024-01-15 11:05:28-03:00,Eu troquei o TNSnames na última vez que deu pr...,41


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [48]:
from langchain.prompts import PromptTemplate

# The initial summary should be under {max_words} words and contain {entity_range} informative Descriptive Entities from the {content_category}.
    # - Your summary should consist of a list of senteces. Each sentence should summarize an information present in the email.
    # - Remember, to keep each summary to max {max_words} words.

prompt_report = PromptTemplate.from_template(
    """
    You are an expert copy-writer that write email conversation reports.
    
    # Background information
    - The emails you will process comprise of conversations of IT Technicians talking about a problem, asking for help, explaninig how to use a technology, or other subjects, etc.
    - You will receive two documents: a report of the previous emails of the conversation and an email of the current conversation.
    - The report and the email are from the same conversation. The email follows the report.
    - If you receive a empty report, this means that the email is the first email of the conversation.
    - Your report will be used to create a knowledge base document of a IT company to be used by IT Technicians. 
            
    # Task
    - You should take the report of the previous conversations and add new information to it from the email. So you will output a new report.
    
    # Guidelines to the email
    - You should not use words from the email that not contain any important information to create a knowledge base document or a manual.
    - At the same time, you should use all the words and information that could be necessary to work with the technologies described in the email and to understande later emails.

    # Guidelines to the new report
    - output just raw text.
    - The new report should combine the information from the previous report with the email.
    - The new report should have all the important information that could be important to understand later emails.
    - Important information to create a knowledge base document: 
            - the problem being addressed, 
            - a exact description char by char of the error message that happened. you should just copy system messages necessary to address the problem.
            - the name of the technology that gave the error message
            - a possible solution, 
            - a technique that didn't work and why it didn't work, 
    - The language of the conversation is generaly in Brazilian Portuguese. So your summary should be in Brazilian Portuguese.
    - The report didn't need to be dense or concise.
    - Dont write uninformative phrases like "the email discusses".
    - Do not discuss the email itself, focus on the content: informative descriptive entities, and details.

    - Some emails dont have any important information or it are just talking about burocratic things like redirecting email to the correct sector, someone acknoledging something, someone thanking, etc. 
        - In this case, you should mantain the previous report as it is

    # Example email:
        '
        Prezada DGBD, boa tarde!

        Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
        dar uma olhada, por favor?


        [2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
        > VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
        > argument(s): "ORA-12541: TNS: nÆo h  listener"

        Atenciosamente,

        Felipe Giacomel
        Divisão de Microinformática
        TRT4 - Secretaria de Tecnologia da Informação e Comunicações
        '

    # Example output summary
        '
        Problema: erro ao utilizar base do Infor de São Leopoldo

        Tecnologia: Oracle

        Mensagem de erro:
        [2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
        > VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
        > argument(s): "ORA-12541: TNS: nÆo h  listener"
        '

    # PREVIOUS REPORT
    {REPORT}
    
    # EMAIL TO BE SUMMARIZED
    {EMAIL_CONTENT}
    """
)

In [49]:
chain_report = prompt_report | llm

In [73]:
from langchain.prompts import PromptTemplate

prompt_report_revisor = PromptTemplate.from_template(
    """
    You are an expert IT Technicians that revises email conversation reports.
    
    # Background information
    - The report you will process is a report of a conversation of IT Technicians talking about a problem, asking for help, explaninig how to use a technology, or other subjects, etc.
    - You will receive two documents: a report of the conversation and an email of the current conversation.
    - The report and the email are from the same conversation.
    - This report will be used to create a knowledge base document of a IT company to be used by IT Technicians. 
            
    # Task
    - Your task is to revise the report to verify if it lacks some important technical information that is present in the email and it is not in the report.
    - You should output a list of the important informations that lack in the report.
    - The informations missing should be only technical information that could be used in the future by the IT Techinicias to understand the system or subject.
    - Remember trivial conversation and questions should be ignored. This type of senteces should not be outputed.    
    
    # Guidelines to the output
    - output just raw text
    - The language of the conversation is generaly in Brazilian Portuguese. So your summary should be in Brazilian Portuguese.
    - Dont write uninformative phrases like "the email discusses".
    - Do not discuss the email itself, focus on the content: informative descriptive entities, and details.

    - Some emails dont have any important technicalinformation or it are just talking about burocratic things like redirecting email to the correct sector, someone acknoledging something, someone thanking, etc. 
        - In this case, you should output empty string.

    # Guidelines of what not output
    - You should not output any text that didn't have any technical information about a system, application.
    - You should not output any text similiar to the following:
            '
            - Eu preciso de ajuda com o aplicativo do Office?
            - João da Silva, Departamento de Computação.
            '
    - REMEMBER!!! IF there is not any important technical information, output a empty string.
            



    # Example of report:
        '
        Problema: erro ao fazer operações na base do Infor de São Leopoldo

        Tecnologia: Oracle

        Mensagem de erro:
        [2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
        > VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
        > argument(s): "ORA-12541: TNS: nÆo h  listener"

        Possível solução: Atualizar o tnsnames.ora na estação de trabalho onde ocorre o erro.

        Técnica que não funcionou: Acessar o banco de dados normalmente e confirmar que o listener está no ar.

        Informação adicional do email:
        O banco de São Leopoldo foi migrado para VMware. Sugere-se atualizar com o mais recente que está na rede.
        '    

    # Example email:
        '
        Felipe, isso não ocorre no servidor do Markvision. O  arquivo atualizado do
        TNSnames  fica no servidor  Heitor, e  todas as estações do TRT têm  acesso
        de leitura.  GPO encarrega de realizar a cópia entre o servidor Heitor para
        as estações.

        Estou acessando as estações que estão ligadas neste momento  em Sleo e até
        agora não encontrei uma que não atualizou o TNSname para versão de
        06/12/2023 e uma tenho capturando para testar.a GPO

        Tem algum chamado sobre este problema ou hosts de estações com problema no
        Infor? Pode ser problema isolado de algumas estações que estão parado ou
        retornaram para a VT


        [image: image.png]
        '

    # Example output
        '
        - O  arquivo atualizado do TNSnames fica no servidor Heitor, e todas as estações do TRT têm  acesso de leitura.
        - Há uma GPO encarregada de realizar a cópia entre o servidor Heitor para as estações.
        '

    # REPORT
    {REPORT}
    
    # EMAIL
    {EMAIL_CONTENT}
    """
)

In [74]:
chain_report_revisor = prompt_report_revisor | llm

In [52]:
from langchain.prompts import PromptTemplate

prompt_combine = PromptTemplate.from_template(
    """
    You are an expert copy-writer that write email conversation reports.
    
    # Background information
    - You will process two documents: a report and a list of informations.
    - The report is a conversation of IT Technicians talking about a problem, asking for help, explaninig how to use a technology, or other subjects, etc, and a
    - Your report will be used to create a knowledge base document of a IT company to be used by IT Technicians. 
            
    # Task
    - You should combine the information of the list with the report.
    
    # Guidelines    
    - You should not remove any information of the report, just add the new information to it. 
    - You will output a new report containing the previous information and the new.
    - You should refraing of change the overall structure of the report. If there isn't a section to put the information, just create a new like '# Informações adicionais'.
    - The language of the conversation is generaly in Brazilian Portuguese. So your summary should be in Brazilian Portuguese.
    - The report didn't need to be dense or concise.
    - Dont write uninformative phrases like "the email discusses".
    - Do not discuss the email itself, focus on the content: informative descriptive entities, and details.
    - If the list of informations is empty, don't output anything.    


    # Example of report:
        '
        Problema: erro ao fazer operações na base do Infor de São Leopoldo

        Tecnologia: Oracle

        Mensagem de erro:
        [2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
        > VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
        > argument(s): "ORA-12541: TNS: nÆo h  listener"

        Possível solução: Atualizar o tnsnames.ora na estação de trabalho onde ocorre o erro.

        Técnica que não funcionou: Acessar o banco de dados normalmente e confirmar que o listener está no ar.

        Informações adicionais:
        - O banco de São Leopoldo foi migrado para VMware
        '    

    # Example list of information:
        '
        - O  arquivo atualizado do TNSnames fica no servidor Heitor, e todas as estações do TRT têm  acesso de leitura.
        - Há uma GPO encarregada de realizar a cópia entre o servidor Heitor para as estações.
        '

    # Example output
        '
        Problema: erro ao fazer operações na base do Infor de São Leopoldo

        Tecnologia: Oracle

        Mensagem de erro:
        [2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
        > VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
        > argument(s): "ORA-12541: TNS: nÆo h  listener"

        Possível solução: Atualizar o tnsnames.ora na estação de trabalho onde ocorre o erro.

        Técnica que não funcionou: Acessar o banco de dados normalmente e confirmar que o listener está no ar.

        Informações adicionais:
        - O banco de São Leopoldo foi migrado para VMware
        - O  arquivo atualizado do TNSnames fica no servidor Heitor, e todas as estações do TRT têm  acesso de leitura.
        - Há uma GPO encarregada de realizar a cópia entre o servidor Heitor para as estações.
        '
    

    # REPORT
    {REPORT}
    
    # LIST OF INFORMATIONS
    {LIST_OF_INFORMATIONS}
    """
)

In [53]:
chain_combine = prompt_combine | llm

In [57]:
def test_prompt(old_report, df_row):
    index, row = df_row
    email = row['body_cleaned']
    
    print( "--------| OLD_REPORT" )
    print( old_report )
    print( "---------------------------------------------" )

    print( "--------| EMAIL" )
    print( email )
    print( "---------------------------------------------" )

    response = chain_report.invoke({'REPORT': old_report, 'EMAIL_CONTENT': email})
    new_report = response.content
    
    print( "--------| NEW_REPORT" )
    print( new_report )
    print( "---------------------------------------------" )

    response = chain_report_revisor.invoke({'REPORT': new_report, 'EMAIL_CONTENT': email})
    list_of_informations = response.content

    print( "--------| LIST_OF_INFORMATIONS" )
    print( list_of_informations )
    print( "---------------------------------------------" )

    response = chain_combine.invoke({'REPORT': new_report, 'LIST_OF_INFORMATIONS': list_of_informations})
    final_report = response.content

    print( "--------| FINAL_REPORT" )
    print( final_report )
    print( "---------------------------------------------" )
    
    return old_report, email, new_report, list_of_informations, final_report


In [66]:
df_iter = df_thread.iterrows()
report = ""
reports = []

In [67]:
old_report, email, new_report, revision, final_report = test_prompt(report, next(df_iter))
reports.append((old_report, email, new_report, revision, final_report))

--------| OLD_REPORT

---------------------------------------------
--------| EMAIL
Prezada DGBD, boa tarde!

Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
dar uma olhada, por favor?


[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
> argument(s): "ORA-12541: TNS: nÆo h  listener"




Atenciosamente,

Felipe Giacomel
Divisão de Microinformática
TRT4 - Secretaria de Tecnologia da Informação e Comunicações

-- 
You received this message because you are subscribed to the Google Groups "SETIC-DIM- Divisao de Microinformatica" group.
To unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.
---------------------------------------------
--------| NEW_REPORT
Problema: erro ao fazer operações na base do Infor de São Leopoldo

Tecnologia: Oracle

Mensagem de erro:
[2024-01-12 11:41:18] Falha ao co

In [75]:
email = """
Prezada DGBD, boa tarde!

Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
dar uma olhada, por favor?


[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
> argument(s): "ORA-12541: TNS: nÆo h  listener"




Atenciosamente,

Felipe Giacomel
Divisão de Microinformática
TRT4 - Secretaria de Tecnologia da Informação e Comunicações

-- 
You received this message because you are subscribed to the Google Groups "SETIC-DIM- Divisao de Microinformatica" group.
To unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.
"""

In [76]:
new_report = """
Problema: erro ao fazer operações na base do Infor de São Leopoldo

Tecnologia: Oracle

Mensagem de erro:
[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
> argument(s): "ORA-12541: TNS: nÆo h  listener"
"""

In [77]:
response = chain_report_revisor.invoke({'REPORT': new_report, 'EMAIL_CONTENT': email})
list_of_informations = response.content

print( "--------| LIST_OF_INFORMATIONS" )
print( list_of_informations )
print( "---------------------------------------------" )

--------| LIST_OF_INFORMATIONS
- Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem dar uma olhada, por favor?
- Prezada DGBD, boa tarde!
- Felipe Giacomel
- Divisão de Microinformática
- TRT4 - Secretaria de Tecnologia da Informação e Comunicações
---------------------------------------------
